In [1]:
import pandas as pd
df = pd.read_csv('../../extractions/dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-l0fy7lkzhf/extraction_result.csv')
df

,source_name,dateline,ingestion_datetime,currency_codes,company_codes_association_ticker_exchange,title,snippet,company_codes_lineage_ticker_exchange,an,company_codes_occur_ticker_exchange,...,document_type,modification_datetime,company_codes,action,region_codes,market_index_codes,company_codes_about_ticker_exchange,company_codes_occur,section,company_codes_relevance_ticker_exchange
0,Interfax: Russia & CIS Business and Financial ...,NaN,1741975207000,NaN,NaN,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,NaN,DAIBN00020250314el3e006y1,NaN,...,article,1741975207000,",rosgos,",add,",asiaz,eeurz,eurz,russ,",NaN,NaN,NaN,NaN,NaN
1,M2 Presswire,NaN,1738055237000,NaN,NaN,Impending fintech consolidation could yield un...,The fintech sector is sustaining its rapid exp...,NaN,MTPW000020250128el1s00001,NaN,...,article,1738055237000,NaN,add,",eurz,uk,weurz,",NaN,NaN,NaN,NaN,NaN
2,Financial News,NaN,1741219207000,NaN,NaN,UK's private share trading plan will be a flop...,The UK's plan to allow trading of private comp...,NaN,LONFIN0020250306el3600001,NaN,...,article,1741219207000,",bttcap,",add,",eurz,uk,weurz,",NaN,NaN,NaN,Private Equity,NaN
3,MFN,NaN,1744351204000,NaN,NaN,VEF initiates a new share buyback program,VEF AB (publ) (“VEF” or the “Company”) has rec...,NaN,MFNEN00020250411el4b0005m,NaN,...,article,1744351204000,",zomhfn,",add,",eecz,eurz,nordz,stock,swed,",NaN,",VEFFF:PSGM,0TX:XFRA,VEFAB:XSTO,",NaN,NaN,NaN
4,CE NoticiasFinancieras,NaN,1739899230000,NaN,NaN,Factop case: Swiss investment fund sues Sauer ...,Nearly two years after the outbreak of the so-...,NaN,NFINCE0020250218el2i008tb,NaN,...,article,1739899230000,NaN,add,",eurz,switz,weurz,",NaN,NaN,NaN,CE Noticias Financieras English,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,Public Companies News and Documents via PUBT,NaN,1739900957000,NaN,NaN,Vinci SA - Determination of the share referenc...,Access the original document here\n\nDetermina...,NaN,LCDVP00020250218el2i00tbo,",VCISF:PSGM,DG:XMIL,VCISY:PINX,DG:XMIL,DG:XPAR...",...,article,1739900957000,",adesmf,banqnp,banqnp,barbip,barbip,crdnat,crd...",add,",eecz,eurz,fra,medz,namz,uk,usa,weurz,",NaN,",VCISF:PSGM,DG:XMIL,VCISY:PINX,DG:XMIL,DG:XPAR...",",snrptb,euruno,euecar,dwitd,crdnat,barbip,banqnp,",NaN,",VCISF:PSGM,DG:XMIL,VCISY:PINX,DG:XMIL,DG:XPAR..."
2907,Public Companies News and Documents via PUBT,NaN,1741884753000,NaN,NaN,Vinpai SA - Details of the Terms and Condition...,Access the original document here\n\nDetails o...,NaN,LCDVP00020250313el3d00yf2,",532834:XBOM,",...,article,1741884753000,",adesmf,adesmf,bobexh,camfcm,camfcm,camfcm,epa...",add,",asiaz,devgcoz,dvpcoz,eecz,eurz,fra,india,medz...",NaN,",532834:XBOM,",",vnpfr,scexcd,euecar,eparlm,camfcm,adesmf,",NaN,",532834:XBOM,"
2908,Oslo Stock Exchange (OSE) Filings,NaN,1740659289000,NaN,NaN,Wilh. Wilhelmsen Holding ASA - Wilh. Wilhelmse...,Access the original document here\n\nWilh. Wil...,NaN,OSEE000020250227el2r000rt,NaN,...,article,1740659289000,",dnc,dnc,dnc,wilhlm,wilhlm,wilhlm,",add,",eurz,namz,nordz,norw,usa,",NaN,NaN,",wilhlm,dnc,",NaN,NaN
2909,Securities and Exchange Commission (SEC) Filings,NaN,1736536946000,NaN,NaN,abrdn Global Dynamic Dividend Fund - Annual Re...,Access the original document here\n\nAnnual Re...,NaN,SAEXC00020250110el1a00ho8,",TJXC:XWBO,1TJX:XMIL,TJX:XFRA,TJXC34:BVMF,0LCE...",...,article,1736537924000,",abvbba,abvbba,adev,adev,agdfun,agdfun,agdfun,...",rep,",cana,caon,eurz,namz,uk,usa,weurz,",NaN,NaN,",zy,zenec,xsleis,wlmsco,vuimth,vmwre,usawas,un...",NaN,",TJXC:XWBO,1TJX:XMIL,TJX:XFRA,TJXC34:BVMF,0LCE..."


## Initial preprocessing

#### Dropping irrelevant cols

In [2]:
cols2drop=['dateline','currency_codes','availability_datetime', 'copyright', 'language_code','region_of_origin','byline','person_codes','publication_date','publisher_name','credit','art','document_type','modification_datetime','market_index_codes'] + [c for c in df.columns if c.endswith('_exchange')]
df = df.drop(columns=cols2drop, errors='ignore') 

In [3]:
## Converting dt
for dt_column in ["modification_date", 'modification_datetime',"ingestion_datetime", "publication_datetime","publication_date"]:
    if dt_column in df.columns:
        df[dt_column] = pd.to_datetime(df[dt_column], unit='ms')

## Aggregating different company codes columns
cc_columns = sorted([c for c in df.columns if c.startswith('company_codes')])
for col in cc_columns:
    df[col] = df[col].apply(lambda x: ','.join(list(set(x.lstrip(',').split(','))-{'',})) if isinstance(x, str) else x)
    df[col] = df[col].fillna("")#.apply(list)
df['all_company_codes'] = df[cc_columns].agg(lambda x: ','.join(x), axis=1).apply(lambda x: ','.join(sorted(list(set(x.split(','))-{''}))))

df = df.drop(columns=cc_columns, errors='ignore') 
df = df.rename(columns={'all_company_codes':'company_codes'})

### Ordering cols logically

In [4]:
columns = ['source_name','title','snippet','body','section','word_count','source_code','industry_codes','company_codes','subject_codes','publication_datetime','ingestion_datetime','modification_date','region_codes','an','action']
assert all([c in df.columns for c in columns]), f"Missing columns: {[c for c in columns if c not in df.columns]}"
df = df[columns]
df

,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action
0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,",c18,c181,c33,c333,cacqu,cactio,ccat,ncat,ncdi...",2025-03-14 16:00:00,2025-03-14 18:00:07,2025-03-15 07:24:00.694,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add
1,M2 Presswire,Impending fintech consolidation could yield un...,The fintech sector is sustaining its rapid exp...,"According to Victor Basta, Managing Partner at...",NaN,621,MTPW,",ifinal,ifmsoft,itech,",,",c18,c181,cacqu,cactio,ccat,ncat,nfact,nfcpin,...",2025-01-28 00:00:00,2025-01-28 09:07:17,2025-01-29 07:25:31.631,",eurz,uk,weurz,",MTPW000020250128el1s00001,add
2,Financial News,UK's private share trading plan will be a flop...,The UK's plan to allow trading of private comp...,The Private Intermittent Securities and Capita...,Private Equity,717,LONFIN,",i81502,i8150203,ialtinv,ifinal,iinv,",bttcap,",c13,c18,c181,cacqu,cactio,ccat,ncat,nfact,nfc...",2025-03-06 00:00:00,2025-03-06 00:00:07,2025-03-10 07:18:14.212,",eurz,uk,weurz,",LONFIN0020250306el3600001,add
3,MFN,VEF initiates a new share buyback program,VEF AB (publ) (“VEF” or the “Company”) has rec...,"Dave Nangle, CEO of VEF comments:\n\n“VEF shar...",NaN,442,MFNEN,",i81502,i8150203,ialtinv,ifinal,iinv,iventure,",zomhfn,",c17,c171,c18,c181,c41,cactio,cboard,ccat,cshb...",2025-04-11 00:00:00,2025-04-11 06:00:04,2025-04-12 07:24:33.453,",eecz,eurz,nordz,stock,swed,",MFNEN00020250411el4b0005m,add
4,CE NoticiasFinancieras,Factop case: Swiss investment fund sues Sauer ...,Nearly two years after the outbreak of the so-...,The vehicle claims to have been defrauded by t...,CE Noticias Financieras English,665,NFINCE,",i81502,ifinal,iinv,",,",c12,c18,c33,c333,cactio,ccat,cliqui,gcat,gcri...",2025-02-18 00:00:00,2025-02-18 17:20:30,2025-02-19 07:28:25.421,",eurz,switz,weurz,",NFINCE0020250218el2i008tb,add
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,Public Companies News and Documents via PUBT,Vinci SA - Determination of the share referenc...,Access the original document here\n\nDetermina...,"Nanterre, 18 February 2025\n\nDetermination of...",NaN,2444,LCDVP,",i831,iconst,icre,ifinal,iinv,","adesmf,banqnp,barbip,crdnat,dwitd,euecar,eurun...",",c17,c172,cactio,ccat,ncat,nfact,nfcpin,npress,",2025-02-18 00:00:00,2025-02-18 17:49:17,2025-02-19 07:29:07.855,",eecz,eurz,fra,medz,namz,uk,usa,weurz,",LCDVP00020250218el2i00tbo,add
2907,Public Companies News and Documents via PUBT,Vinpai SA - Details of the Terms and Condition...,Access the original document here\n\nDetails o...,* Camlin Fine Science Limited and certain sign...,NaN,2333,LCDVP,",i25,i41,i831,i83106,ibasicm,icnp,ifinal,ifood...","adesmf,bobexh,camfcm,eparlm,euecar,nseoil,scex...",",c18,c181,cacqu,cactio,ccat,ctendo,ncat,nfact,...",2025-03-13 00:00:00,2025-03-13 16:52:33,2025-03-14 07:34:31.870,",asiaz,devgcoz,dvpcoz,eecz,eurz,fra,india,medz...",LCDVP00020250313el3d00yf2,add
2908,Oslo Stock Exchange (OSE) Filings,Wilh. Wilhelmsen Holding ASA - Wilh. Wilhelmse...,Access the original document here\n\nWilh. Wil...,Wilh. Wilhelmsen Holding ASA: Share buy back p...,NaN,768,OSEE,",i74,i814,i81402,ibnk,ifinal,iship,itsp,","dnc,wilhlm",",c17,c171,c18,c181,cactio,ccat,cgvfil,cshbuy,n...",2025-02-27 00:00:00,2025-02-27 12:28:09,2025-02-28 07:20:52.620,",eurz,namz,nordz,norw,usa,",OSEE000020250227el2r000rt,add
2909,Securities and Exchange Commission (SEC) Filings,abrdn Global Dynamic Dividend Fund - Annual Re...,Access the original document here\n\nAnnual Re...,UNITED STATES\n\nSECURITIES AND EXCHANGE COMMI...,NaN,71884,SAEXC,",i81502,i8150206,i831,i83107,ifinal,iinv,","abvbba,adev,agdfun,altdd

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2911 entries, 0 to 2910
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   source_name           2911 non-null   object        
 1   title                 2911 non-null   object        
 2   snippet               2834 non-null   object        
 3   body                  2911 non-null   object        
 4   section               195 non-null    object        
 5   word_count            2911 non-null   int64         
 6   source_code           2911 non-null   object        
 7   industry_codes        2911 non-null   object        
 8   company_codes         2911 non-null   object        
 9   subject_codes         2911 non-null   object        
 10  publication_datetime  2911 non-null   datetime64[ns]
 11  ingestion_datetime    2911 non-null   datetime64[ns]
 12  modification_date     2911 non-null   datetime64[ns]
 13  region_codes      

### Decoding company names

In [6]:
from factiva.news.taxonomy import Taxonomy
user_key = 'LKbI9fY6ZEPU8RcJUxQHjwKbld52WGt0'
taxonomy = Taxonomy(user_key)
company_codes_df = taxonomy.get_category_codes('companies')
company_codes_df = company_codes_df[~company_codes_df.index.duplicated(keep='first')]
company_codes_df

,description,ticker,ticker_exchange,exchange,primary_exchange
code,,,,,
HANY,Harmony-DML,NaN,NaN,NaN,NaN
VTKSPL,Viatek Services Pty Ltd,NaN,NaN,NaN,NaN
IIPAUS,IMPRESSIONS IN PRINT,NaN,NaN,NaN,NaN
OITRAR,OIL-TRAST OOO,NaN,NaN,NaN,NaN
DNGHCL,Donghee Holdings Co. Ltd.,NaN,NaN,NaN,NaN
...,...,...,...,...,...
BERTO,Bertocchi Smallgoods,NaN,NaN,NaN,NaN
GLBWIN,GLOBAL WINES,NaN,NaN,NaN,NaN
RNGASW,Ringier Ag,NaN,NaN,NaN,NaN


In [7]:
company_dict = company_codes_df.description.to_dict()
def extract_company_names(company_codes_list):
    company_codes_list = company_codes_list.upper().split(',')
    
    company_names = []
    for code in company_codes_list:
        if code in company_dict.keys():
            company_name = company_dict.get(code,code)
            company_names.append(company_name)
        else:
            company_names.append(code)
    return ','.join(company_names)

df['company_names'] = df['company_codes'].apply(lambda x: extract_company_names(x))
df

,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action,company_names
0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,",c18,c181,c33,c333,cacqu,cactio,ccat,ncat,ncdi...",2025-03-14 16:00:00,2025-03-14 18:00:07,2025-03-15 07:24:00.694,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add,Central Bank of the Russian Federation
1,M2 Presswire,Impending fintech consolidation could yield un...,The fintech sector is sustaining its rapid exp...,"According to Victor Basta, Managing Partner at...",NaN,621,MTPW,",ifinal,ifmsoft,itech,",,",c18,c181,cacqu,cactio,ccat,ncat,nfact,nfcpin,...",2025-01-28 00:00:00,2025-01-28 09:07:17,2025-01-29 07:25:31.631,",eurz,uk,weurz,",MTPW000020250128el1s00001,add,
2,Financial News,UK's private share trading plan will be a flop...,The UK's plan to allow trading of private comp...,The Private Intermittent Securities and Capita...,Private Equity,717,LONFIN,",i81502,i8150203,ialtinv,ifinal,iinv,",bttcap,",c13,c18,c181,cacqu,cactio,ccat,ncat,nfact,nfc...",2025-03-06 00:00:00,2025-03-06 00:00:07,2025-03-10 07:18:14.212,",eurz,uk,weurz,",LONFIN0020250306el3600001,add,Better Capital LLP
3,MFN,VEF initiates a new share buyback program,VEF AB (publ) (“VEF” or the “Company”) has rec...,"Dave Nangle, CEO of VEF comments:\n\n“VEF shar...",NaN,442,MFNEN,",i81502,i8150203,ialtinv,ifinal,iinv,iventure,",zomhfn,",c17,c171,c18,c181,c41,cactio,cboard,ccat,cshb...",2025-04-11 00:00:00,2025-04-11 06:00:04,2025-04-12 07:24:33.453,",eecz,eurz,nordz,stock,swed,",MFNEN00020250411el4b0005m,add,VEF AB
4,CE NoticiasFinancieras,Factop case: Swiss investment fund sues Sauer ...,Nearly two years after the outbreak of the so-...,The vehicle claims to have been defrauded by t...,CE Noticias Financieras English,665,NFINCE,",i81502,ifinal,iinv,",,",c12,c18,c33,c333,cactio,ccat,cliqui,gcat,gcri...",2025-02-18 00:00:00,2025-02-18 17:20:30,2025-02-19 07:28:25.421,",eurz,switz,weurz,",NFINCE0020250218el2i008tb,add,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,Public Companies News and Documents via PUBT,Vinci SA - Determination of the share referenc...,Access the original document here\n\nDetermina...,"Nanterre, 18 February 2025\n\nDetermination of...",NaN,2444,LCDVP,",i831,iconst,icre,ifinal,iinv,","adesmf,banqnp,barbip,crdnat,dwitd,euecar,eurun...",",c17,c172,cactio,ccat,ncat,nfact,nfcpin,npress,",2025-02-18 00:00:00,2025-02-18 17:49:17,2025-02-19 07:29:07.855,",eecz,eurz,fra,medz,namz,uk,usa,weurz,",LCDVP00020250218el2i00tbo,add,"Autorité des Marchés Financiers (France),BNP P..."
2907,Public Companies News and Documents via PUBT,Vinpai SA - Details of the Terms and Condition...,Access the original document here\n\nDetails o...,* Camlin Fine Science Limited and certain sign...,NaN,2333,LCDVP,",i25,i41,i831,i83106,ibasicm,icnp,ifinal,ifood...","adesmf,bobexh,camfcm,eparlm,euecar,nseoil,scex...",",c18,c181,cacqu,cactio,ccat,ctendo,ncat,nfact,...",2025-03-13 00:00:00,2025-03-13 16:52:33,2025-03-14 07:34:31.870,",asiaz,devgcoz,dvpcoz,eecz,eurz,fra,india,medz...",LCDVP00020250313el3d00yf2,add,"Autorité des Marchés Financiers (France),BSE L..."
2908,Oslo Stock Exchange (OSE) Filings,Wilh. Wilhelmsen Holding ASA - Wilh. Wilhelmse...,Access the original document here\n\nWilh. Wil...,Wilh. Wilhelmsen Holding ASA: Share buy back p...,NaN,768,OSEE,",i74,i814,i81402,ibnk,ifinal,iship,itsp,","dnc,wilhlm",",c17,c171,c18,c181,cactio,ccat,cgvfil,cshbuy,n...",2025-02-27 00:00:00,2025-02-27 12:28:09,2025-02-28 07:20:52.620,",eurz,namz,nordz,norw,usa,",OSEE000020250227el2r000rt,add,"DnB Bank ASA,Wilhelm Wilhelmsen Holding ASA"
2909,Securities and Exchange Commission (SEC) Filings,

### Decoding industries

In [8]:
industries_df = taxonomy.get_category_codes('industries')
industries_mapping_dict = industries_df.description.to_dict()

def extract_industries(industry_codes_list):
    industry_codes_list = sorted(list(set(industry_codes_list.split(','))-{''}))
    
    industry_descriptions = []
    for code in industry_codes_list:
        if code in industries_mapping_dict.keys():
            industry_desc = industries_mapping_dict.get(code,code)
            industry_descriptions.append(industry_desc)
        else:
            industry_descriptions.append(code)
    return ','.join(industry_descriptions)
df['industries'] = df['industry_codes'].apply(lambda x: extract_industries(x))
df

,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action,company_names,industries
0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,",c18,c181,c33,c333,cacqu,cactio,ccat,ncat,ncdi...",2025-03-14 16:00:00,2025-03-14 18:00:07,2025-03-15 07:24:00.694,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add,Central Bank of the Russian Federation,"Energy,Coal Mining,Primary Metals,Non-ferrous ..."
1,M2 Presswire,Impending fintech consolidation could yield un...,The fintech sector is sustaining its rapid exp...,"According to Victor Basta, Managing Partner at...",NaN,621,MTPW,",ifinal,ifmsoft,itech,",,",c18,c181,cacqu,cactio,ccat,ncat,nfact,nfcpin,...",2025-01-28 00:00:00,2025-01-28 09:07:17,2025-01-29 07:25:31.631,",eurz,uk,weurz,",MTPW000020250128el1s00001,add,,"Financial Services,Financial Technology,Techno..."
2,Financial News,UK's private share trading plan will be a flop...,The UK's plan to allow trading of private comp...,The Private Intermittent Securities and Capita...,Private Equity,717,LONFIN,",i81502,i8150203,ialtinv,ifinal,iinv,",bttcap,",c13,c18,c181,cacqu,cactio,ccat,ncat,nfact,nfc...",2025-03-06 00:00:00,2025-03-06 00:00:07,2025-03-10 07:18:14.212,",eurz,uk,weurz,",LONFIN0020250306el3600001,add,Better Capital LLP,"Trusts/Funds/Financial Vehicles,Private Equity..."
3,MFN,VEF initiates a new share buyback program,VEF AB (publ) (“VEF” or the “Company”) has rec...,"Dave Nangle, CEO of VEF comments:\n\n“VEF shar...",NaN,442,MFNEN,",i81502,i8150203,ialtinv,ifinal,iinv,iventure,",zomhfn,",c17,c171,c18,c181,c41,cactio,cboard,ccat,cshb...",2025-04-11 00:00:00,2025-04-11 06:00:04,2025-04-12 07:24:33.453,",eecz,eurz,nordz,stock,swed,",MFNEN00020250411el4b0005m,add,VEF AB,"Trusts/Funds/Financial Vehicles,Private Equity..."
4,CE NoticiasFinancieras,Factop case: Swiss investment fund sues Sauer ...,Nearly two years after the outbreak of the so-...,The vehicle claims to have been defrauded by t...,CE Noticias Financieras English,665,NFINCE,",i81502,ifinal,iinv,",,",c12,c18,c33,c333,cactio,ccat,cliqui,gcat,gcri...",2025-02-18 00:00:00,2025-02-18 17:20:30,2025-02-19 07:28:25.421,",eurz,switz,weurz,",NFINCE0020250218el2i008tb,add,,"Trusts/Funds/Financial Vehicles,Financial Serv..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,Public Companies News and Documents via PUBT,Vinci SA - Determination of the share referenc...,Access the original document here\n\nDetermina...,"Nanterre, 18 February 2025\n\nDetermination of...",NaN,2444,LCDVP,",i831,iconst,icre,ifinal,iinv,","adesmf,banqnp,barbip,crdnat,dwitd,euecar,eurun...",",c17,c172,cactio,ccat,ncat,nfact,nfcpin,npress,",2025-02-18 00:00:00,2025-02-18 17:49:17,2025-02-19 07:29:07.855,",eecz,eurz,fra,medz,namz,uk,usa,weurz,",LCDVP00020250218el2i00tbo,add,"Autorité des Marchés Financiers (France),BNP P...","Financial Investment Services,Construction,Rea..."
2907,Public Companies News and Documents via PUBT,Vinpai SA - Details of the Terms and Condition...,Access the original document here\n\nDetails o...,* Camlin Fine Science Limited and certain sign...,NaN,2333,LCDVP,",i25,i41,i831,i83106,ibasicm,icnp,ifinal,ifood...","adesmf,bobexh,camfcm,eparlm,euecar,nseoil,scex...",",c18,c181,cacqu,cactio,ccat,ctendo,ncat,nfact,...",2025-03-13 00:00:00,2025-03-13 16:52:33,2025-03-14 07:34:31.870,",asiaz,devgcoz,dvpcoz,eecz,eurz,fra,india,medz...",LCDVP00020250313el3d00yf2,add,"Autorité des Marchés Financiers (France),BSE L...","Chemicals,Food/Beverages,Financial Investment ..."
2908,Oslo Stock Exchange (OSE) Filings,Wilh. Wilhelmsen Holding ASA - Wilh. Wilhelmse...,Access the original document here\n\nWilh. Wil...,Wilh. Wilhelmse

### Improvements: 
* Splitting very large texts into smaller articles
* Cleaning dataset (ads, HTML code)
* Removing Duplicates / very similar articles

In [9]:
df

,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action,company_names,industries
0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,",c18,c181,c33,c333,cacqu,cactio,ccat,ncat,ncdi...",2025-03-14 16:00:00,2025-03-14 18:00:07,2025-03-15 07:24:00.694,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add,Central Bank of the Russian Federation,"Energy,Coal Mining,Primary Metals,Non-ferrous ..."
1,M2 Presswire,Impending fintech consolidation could yield un...,The fintech sector is sustaining its rapid exp...,"According to Victor Basta, Managing Partner at...",NaN,621,MTPW,",ifinal,ifmsoft,itech,",,",c18,c181,cacqu,cactio,ccat,ncat,nfact,nfcpin,...",2025-01-28 00:00:00,2025-01-28 09:07:17,2025-01-29 07:25:31.631,",eurz,uk,weurz,",MTPW000020250128el1s00001,add,,"Financial Services,Financial Technology,Techno..."
2,Financial News,UK's private share trading plan will be a flop...,The UK's plan to allow trading of private comp...,The Private Intermittent Securities and Capita...,Private Equity,717,LONFIN,",i81502,i8150203,ialtinv,ifinal,iinv,",bttcap,",c13,c18,c181,cacqu,cactio,ccat,ncat,nfact,nfc...",2025-03-06 00:00:00,2025-03-06 00:00:07,2025-03-10 07:18:14.212,",eurz,uk,weurz,",LONFIN0020250306el3600001,add,Better Capital LLP,"Trusts/Funds/Financial Vehicles,Private Equity..."
3,MFN,VEF initiates a new share buyback program,VEF AB (publ) (“VEF” or the “Company”) has rec...,"Dave Nangle, CEO of VEF comments:\n\n“VEF shar...",NaN,442,MFNEN,",i81502,i8150203,ialtinv,ifinal,iinv,iventure,",zomhfn,",c17,c171,c18,c181,c41,cactio,cboard,ccat,cshb...",2025-04-11 00:00:00,2025-04-11 06:00:04,2025-04-12 07:24:33.453,",eecz,eurz,nordz,stock,swed,",MFNEN00020250411el4b0005m,add,VEF AB,"Trusts/Funds/Financial Vehicles,Private Equity..."
4,CE NoticiasFinancieras,Factop case: Swiss investment fund sues Sauer ...,Nearly two years after the outbreak of the so-...,The vehicle claims to have been defrauded by t...,CE Noticias Financieras English,665,NFINCE,",i81502,ifinal,iinv,",,",c12,c18,c33,c333,cactio,ccat,cliqui,gcat,gcri...",2025-02-18 00:00:00,2025-02-18 17:20:30,2025-02-19 07:28:25.421,",eurz,switz,weurz,",NFINCE0020250218el2i008tb,add,,"Trusts/Funds/Financial Vehicles,Financial Serv..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,Public Companies News and Documents via PUBT,Vinci SA - Determination of the share referenc...,Access the original document here\n\nDetermina...,"Nanterre, 18 February 2025\n\nDetermination of...",NaN,2444,LCDVP,",i831,iconst,icre,ifinal,iinv,","adesmf,banqnp,barbip,crdnat,dwitd,euecar,eurun...",",c17,c172,cactio,ccat,ncat,nfact,nfcpin,npress,",2025-02-18 00:00:00,2025-02-18 17:49:17,2025-02-19 07:29:07.855,",eecz,eurz,fra,medz,namz,uk,usa,weurz,",LCDVP00020250218el2i00tbo,add,"Autorité des Marchés Financiers (France),BNP P...","Financial Investment Services,Construction,Rea..."
2907,Public Companies News and Documents via PUBT,Vinpai SA - Details of the Terms and Condition...,Access the original document here\n\nDetails o...,* Camlin Fine Science Limited and certain sign...,NaN,2333,LCDVP,",i25,i41,i831,i83106,ibasicm,icnp,ifinal,ifood...","adesmf,bobexh,camfcm,eparlm,euecar,nseoil,scex...",",c18,c181,cacqu,cactio,ccat,ctendo,ncat,nfact,...",2025-03-13 00:00:00,2025-03-13 16:52:33,2025-03-14 07:34:31.870,",asiaz,devgcoz,dvpcoz,eecz,eurz,fra,india,medz...",LCDVP00020250313el3d00yf2,add,"Autorité des Marchés Financiers (France),BSE L...","Chemicals,Food/Beverages,Financial Investment ..."
2908,Oslo Stock Exchange (OSE) Filings,Wilh. Wilhelmsen Holding ASA - Wilh. Wilhelmse...,Access the original document here\n\nWilh. Wil...,Wilh. Wilhelmse

## Splitting long articles
* Given volume and potential hardware limitations/speed considerations - TextTilingTokenizer seems [a good choice](https://medium.com/blue-orange-digital/document-chunking-guided-journey-a-greedy-start-e39a1dff15c3)

In [ ]:
## requirements: tqdm-joblib, nltk, 

In [ ]:
import re
import os

from bs4 import BeautifulSoup
from nltk.tokenize import TextTilingTokenizer
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib 
from tqdm.autonotebook import tqdm
tqdm.auto()

TOKEN_RX = re.compile(r"[A-Za-z0-9]+(?:'[A-Za-z0-9]+)?")

def factiva_wordcount(html: str) -> int:
    """
    Approximate Factiva 'WC' very closely (±0–2 % on real wires).
    1. Strip markup quickly with lxml‑backed BeautifulSoup.
    2. Replace hyphens & slashes with space so they split tokens.
    3. Count regex‑defined tokens.
    """
    # -------- strip HTML completely --------
    txt = BeautifulSoup(html, "lxml").get_text(" ")

    # -------- normalise separators ---------
    txt = txt.replace("‑", "-")           # unicode hyphen
    txt = re.sub(r"[/-]", " ", txt)       # split hyphen/slash compounds
    txt = txt.replace("\u00A0", " ")      # non‑breaking sp

    # -------- tokenise & count ------------
    return len(TOKEN_RX.findall(txt))



bodies  = df["body"].tolist()    
CPUS    = os.cpu_count()

tokenizer = TextTilingTokenizer()

def _tile(doc):
    try:
        result = tokenizer.tokenize(doc)
        return result
    except ValueError as e:
        print(f"Error processing document: {e}, len of doc: {factiva_wordcount(doc)}, document: {doc}")
        return [doc]                                   

with tqdm_joblib(tqdm(desc="Tiling", total=len(bodies))):# and store the results in a new column
    df["segments"] = Parallel(n_jobs=CPUS, backend="multiprocessing",batch_size=4)(delayed(_tile)(doc) for doc in bodies)
    
df['num_segments'] = df['segments'].apply(lambda x: len(x))
df_exploded = df.explode('segments')


### Hashing and dedup
* How to drop duplicates without over-relying on Transformers? -> all sorts of hashing


In [ ]:
## requirements: datasketch, simhash
import hashlib

from collections import defaultdict
from typing import Dict, Iterable, List, Set

from datasketch import MinHash, MinHashLSH
from simhash import Simhash, SimhashIndex


re_ws  = re.compile(r"\s+")

df_exploded["segment_norm"] = (
    df_exploded["segments"]
      .str.lower()
      .str.replace(re_ws, " ", regex=True)   # regex compiled once in C
      .str.strip()
)

# 2 — hash (still a Python loop but the strings are now short & tidy)
df_exploded["segment_hash"] = df_exploded["segment_norm"].map(
        lambda t: hashlib.md5(t.encode()).hexdigest()
)
print(df_exploded['segment_hash'].nunique(), df_exploded['segments'].nunique(), df_exploded['segments'].nunique()/df_exploded['segment_hash'].nunique())
df_exploded.drop_duplicates(subset=['segment_hash'], inplace=True)
df_exploded.reset_index(inplace=True)
df_exploded

In [ ]:
def shingles(text: str, n: int = 8) -> Iterable[str]:
    """Yield n‑word shingles (lower‑cased, non‑word chars collapsed)."""
    tokens = re.sub(r"\W+", " ", text).lower().split()
    for i in range(max(0, len(tokens) - n + 1)):
        yield " ".join(tokens[i : i + n])


def union_find(n: int):
    """Classic union‑find / disjoint‑set with path compression & union‑by‑size."""
    parent = list(range(n))
    size = [1] * n

    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union(a, b):
        ra, rb = find(a), find(b)
        if ra == rb:
            return
        if size[ra] < size[rb]:
            ra, rb = rb, ra
        parent[rb] = ra
        size[ra] += size[rb]

    return parent, find, union

def minhash_dedup(
    df: pd.DataFrame,
    col: str = "segment_norm",
    num_perm: int = 128,
    jaccard_thresh: float = 0.9,
) -> Dict[int, List[int]]:
    """Return clusters {root_idx: [member_idxs …]} by MinHash‑LSH.@≥thresh"""

    lsh = MinHashLSH(threshold=jaccard_thresh, num_perm=num_perm)
    mhs: List[MinHash] = []

    # 0‑based reindex for contiguous union‑find
    df = df.reset_index(drop=True)

    for idx, txt in tqdm(df[col].items(), desc="MinHash: build", unit="doc"):
        mh = MinHash(num_perm=num_perm)
        for sh in shingles(txt, n=8):
            mh.update(sh.encode())
        lsh.insert(str(idx), mh)
        mhs.append(mh)

    parent, find, union = union_find(len(mhs))

    for idx, mh in tqdm(enumerate(mhs), total=len(mhs), desc="MinHash: query", unit="doc"):
        for candidate in lsh.query(mh):
            j = int(candidate)
            if j <= idx:
                continue  # avoid self / duplicate pairs
            union(idx, j)

    clusters: Dict[int, List[int]] = defaultdict(list)
    for i in range(len(mhs)):
        clusters[find(i)].append(i)
    return clusters


def containment_dedup(
    df: pd.DataFrame,
    candidates: Iterable[int],
    col: str = "segment_norm",
) -> Set[int]:
    """Return the indices **to keep** after removing verbatim substrings.

    Strategy: sort survivors by length; for each *longer* doc scan shorter ones.
    Optional optimisation with Aho‑Corasick if pyahocorasick is installed.
    """

    try:
        import ahocorasick

        automaton = ahocorasick.Automaton()
        texts = {i: df.at[i, col] for i in candidates}
        # shorter → pattern, longer → search text
        by_len = sorted(texts.items(), key=lambda kv: len(kv[1]))
        keep: Set[int] = set(texts.keys())
        for i, (idx_i, pat) in enumerate(by_len):
            automaton.add_word(pat, (idx_i, pat))
        automaton.make_automaton()

        for idx_long, txt_long in reversed(by_len):
            for end_idx, (idx_short, _pat) in automaton.iter(txt_long):
                if idx_short == idx_long or idx_short not in keep:
                    continue
                keep.discard(idx_short)  # short text fully contained
        return keep

    except ImportError:
        # fallback O(n^2) – OK for ≤100k survivors
        texts = {i: df.at[i, col] for i in candidates}
        by_len = sorted(texts.items(), key=lambda kv: len(kv[1]))
        keep: Set[int] = set(texts.keys())
        for j in range(len(by_len)):
            idx_short, short_txt = by_len[j]
            if idx_short not in keep:
                continue
            for idx_long, long_txt in by_len[j + 1 :]:
                if idx_long not in keep:
                    continue
                if short_txt in long_txt:
                    keep.discard(idx_short)
                    break
        return keep
    
def simhash_dedup(
    df: pd.DataFrame,
    candidates: Iterable[int],
    col: str = "segment_norm",
    ngram: int = 4,
    hamming_thresh: int = 3,
) -> Set[int]:
    """Return indices to keep after near‑dup removal via SimHash64."""

    def tok4(text: str) -> List[str]:
        toks = re.sub(r"\W+", " ", text).lower().split()
        if len(toks) < ngram:
            return toks
        return [" ".join(toks[i : i + ngram]) for i in range(len(toks) - ngram + 1)]

    # Build SimhashIndex
    objs = [(str(i), Simhash(tok4(df.at[i, col]), f=64)) for i in candidates]
    index = SimhashIndex(objs, k=hamming_thresh)

    dup_ids: Set[int] = set()
    for doc_id, sim in objs:
        if int(doc_id) in dup_ids:
            continue
        near = index.get_near_dups(sim)
        for d in near:
            if d != doc_id:
                dup_ids.add(int(d))
    keep = {i for i in candidates if i not in dup_ids}
    return keep


In [ ]:
def deduplicate(df: pd.DataFrame, col: str = "segment_norm") -> pd.DataFrame:
    """Full three‑stage dedup; returns a *new* DataFrame of unique docs."""

    print("[1/3] MinHash clustering …")
    clusters = minhash_dedup(df, col)
    canon_indices = {
        max(idxs, key=lambda i: len(df.at[i, col])) for idxs in clusters.values()
    }

    print("[2/3] Substring containment filter …")
    canon_indices = containment_dedup(df, canon_indices, col)

    print("[3/3] SimHash fuzzy filter …")
    canon_indices = simhash_dedup(df, canon_indices, col)

    return df.loc[sorted(canon_indices)].reset_index(drop=True)

In [18]:
df_deduped = deduplicate(df_exploded, col="segment_norm")

[1/3] MinHash clustering …


MinHash: build: 0doc [00:00, ?doc/s]

MinHash: query:   0%|          | 0/58223 [00:00<?, ?doc/s]

[2/3] Substring containment filter …
[3/3] SimHash fuzzy filter …


Big bucket found. key:20:0, len:209
Big bucket found. key:f950:0, len:241
Big bucket found. key:2341:1, len:276
Big bucket found. key:20:0, len:209
Big bucket found. key:f950:0, len:241
Big bucket found. key:f950:0, len:241
Big bucket found. key:20:0, len:209
Big bucket found. key:df33:3, len:220
Big bucket found. key:20:0, len:209
Big bucket found. key:20:0, len:209
Big bucket found. key:20:0, len:209
Big bucket found. key:f950:0, len:241
Big bucket found. key:20:0, len:209
Big bucket found. key:20:0, len:209
Big bucket found. key:d580:3, len:201
Big bucket found. key:20:0, len:209
Big bucket found. key:20:0, len:209
Big bucket found. key:2341:1, len:276
Big bucket found. key:f950:0, len:241
Big bucket found. key:df33:3, len:220
Big bucket found. key:d580:3, len:201
Big bucket found. key:20:0, len:209
Big bucket found. key:f950:0, len:241
Big bucket found. key:df33:3, len:220
Big bucket found. key:f950:0, len:241
Big bucket found. key:2341:1, len:276
Big bucket found. key:20:0, len:20

In [21]:
df_deduped['segment_word_count'] = df_deduped['segments'].progress_apply(lambda x: factiva_wordcount(x))
df_deduped

  0%|          | 0/42511 [00:00<?, ?it/s]

,index,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,...,region_codes,an,action,company_names,industries,segments,num_segments,segment_norm,segment_hash,segment_word_count
0,0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,...,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add,Central Bank of the Russian Federation,"Energy,Coal Mining,Primary Metals,Non-ferrous ...",*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,14,*** russia's broad monetary base declines 2.8%...,d0002cdd92d6cbcb0f256ca3dd01b52f,160
1,0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,...,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add,Central Bank of the Russian Federation,"Energy,Coal Mining,Primary Metals,Non-ferrous ...",\n\nThe Central Bank of Russia (CBR) is propos...,14,the central bank of russia (cbr) is proposing ...,87aa3b8cb4adf24da8c26faefe77e450,65
2,0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,...,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add,Central Bank of the Russian Federation,"Energy,Coal Mining,Primary Metals,Non-ferrous ...",\n\n*** CENTRAL BANK RAISES 2025 PROFIT FORECA...,14,*** central bank raises 2025 profit forecast f...,d84083425756fc7404d418985ba30081,145
3,0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,...,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add,Central Bank of the Russian Federation,"Energy,Coal Mining,Primary Metals,Non-ferrous ...","\n\n""He [Energy Minister Sergei Tsivilev] said...",14,"""he [energy minister sergei tsivilev] said tha...",b5897c61d8106f2345c1434504aed73b,236
4,0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,...,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add,Central Bank of the Russian Federation,"Energy,Coal Mining,Primary Metals,Non-ferrous ...",\n\n*** FOUNDER OF SIBERIAN BUSINESS UNION HOL...,14,*** founder of siberian business union holding...,35861e069c32bd473636801a640717b7,196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42506,2909,Securities and Exchange Commission (SEC) Filings,abrdn Global Dynamic Dividend Fund - Annual Re...,Access the original document here\n\nAnnual Re...,UNITED STATES\n\nSECURITIES AND EXCHANGE COMMI...,NaN,71884,SAEXC,",i81502,i8150206,i831,i83107,ifinal,iinv,","abvbba,adev,agdfun,altddf,amade,amdcs,amtowc,a...",...,",cana,caon,eurz,namz,uk,usa,weurz,",SAEXC00020250110el1a00ho8,rep,"AbbVie Inc.,Analog Devices Inc,abrdn Global Dy...","Trusts/Funds/Financial Vehicles,Investment Tru...","\n\nExcept for cross-hedges, the Fund will not...",209,"except for cross-hedges, the fund will not ent...",d5fc4218a104a6480767a5dfc0090d99,181
42507,2909,Securities and Exchange Commission (SEC) Filings,abrdn Global Dynamic Dividend Fund - Annual Re...,Access the original document here\n\nAnnual Re...,UNITED STATES\n\nSECURITIES AND EXCHANGE COMMI...,NaN,71884,SAEXC,",i81502,i8150206,i831

## Exporting results

In [ ]:
df.to_csv('../../extractions/dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-l0fy7lkzhf/preprocessing_result.csv', index=False, encoding='utf-8')

In [ ]:
df_deduped.rename(columns={'index':'original_index'}, inplace=True)
df_deduped.to_csv('../../extractions/dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-l0fy7lkzhf/preprocessing_result_deduped.csv', index=False, encoding='utf-8')